In [7]:
import numpy as np
import pandas as pd

from desdeo_problem.testproblems.TestProblems import test_problem_builder

from desdeo_emo.EAs.NIMBUS_EA import NIMBUS_RVEA, NIMBUS_NSGAIII
from desdeo_emo.EAs.NSGAIII import NSGAIII
from desdeo_emo.EAs.RVEA import RVEA
import plotly.express as ex
from pygmo import fast_non_dominated_sorting as nds
import plotly.graph_objects as go

In [8]:
problem = test_problem_builder(name="DTLZ7", n_of_objectives=3, n_of_variables=5)

In [11]:
wholefront = NSGAIII(problem)

In [12]:
while wholefront.continue_evolution():
    wholefront.iterate()

In [15]:
ex.scatter_3d(x=wholefront.population.objectives[:,0],y=wholefront.population.objectives[:,1],z=wholefront.population.objectives[:,2])

In [18]:
problem.ideal = wholefront.population.objectives.min(axis=0)
problem.nadir = wholefront.population.objectives.max(axis=0)

In [19]:
evolver = NIMBUS_NSGAIII(problem, n_gen_per_iter=200)

In [20]:
plot, pref = evolver.requests()

In [21]:
print(pref.content)

{'dimensions_data':                 f1        f2       f3
minimize         1         1        1
ideal            0         0  2.74146
nadir     0.887924  0.885135        6, 'message': 'Provide a reference point worse than to the ideal point and better than the nadir point.\nIdeal point: \nf1          0\nf2          0\nf3    2.74146\nName: ideal, dtype: object\nNadir point: \nf1    0.887924\nf2    0.885135\nf3           6\nName: nadir, dtype: object\nThe reference point will be used to create scalarization functions in the preferred region.\n', 'validator': <function validate_ref_point_with_ideal_and_nadir at 0x7f2760ebfe60>}


In [22]:
response = np.asarray([[0.4,0.2, 3]])
pref.response = pd.DataFrame(response, columns=pref.content['dimensions_data'].columns)

In [23]:
plot, pref = evolver.iterate(pref)

In [27]:
fig = plot3d(plot, response)
fig

In [26]:
def plot3d(plot, response):
    # NIMBUSEA
    obj = plot.content['data'].values
    non_dom_id = nds(obj)[0][0]
    fig = go.Figure(
        go.Scatter3d(
            x=obj[non_dom_id, 0],
            y=obj[non_dom_id, 1],
            z=obj[non_dom_id, 2],
            name="NIMBUS-RVEA",
            mode='markers',
            marker=dict(
            size=4,
            opacity=0.8
        )
        )
    )

    #True Front
    fig.add_trace(
        go.Scatter3d(
            x=wholefront.population.objectives[:,0],
            y=wholefront.population.objectives[:,1],
            z=wholefront.population.objectives[:,2],
            name="True Front",
            marker=dict(
            size=2,
            opacity=0.8
        )
        )
    )

    #Nadir Point
    fig.add_trace(
        go.Scatter3d(
            x=[wholefront.population.objectives.max(axis=0)[0]],
            y=[wholefront.population.objectives.max(axis=0)[1]],
            z=[wholefront.population.objectives.max(axis=0)[2]],
            name="Nadir Point",
            marker=dict(
            size=4,
            opacity=0.8
        )
        )
    )



    # Reference Point

    fig.add_trace(
        go.Scatter3d(
            x=[response[0,0]],
            y=[response[0,1]],
            z=[response[0,2]],
            name="Reference Point",
            marker=dict(
            size=4,
            opacity=0.8
        )
        )
    )

    # STOM

    fig.add_trace(
        go.Scatter3d(
            x=[0, response[0,0],100*response[0,0]],
            y=[0, response[0,1],100*response[0,1]],
            z=[0, response[0,2],100*response[0,2]],
            name="STOM",
            marker=dict(
            size=2,
            opacity=0.8
        ),
            line=dict(width=10)
        )
    )

    #ASF

    fig.add_trace(
        go.Scatter3d(
            x=[response[0,0]-1, response[0,0]+1],
            y=[response[0,1]-1, response[0,1]+1],
            z=[response[0,2]-1, response[0,2]+1],
            name="ASF",
            marker=dict(
            size=2,
            opacity=0.8
        ),
            line=dict(width=10)
        )
    )
    #AUG

    fig.add_trace(
        go.Scatter3d(
            x=[10*response[0,0] - 9*0.5,response[0,0], 0.5],
            y=[10*response[0,1] - 9*0.5,response[0,1], 0.5],
            z=[10*response[0,2] - 9*0.5,response[0,2], 0.5],
            name="AUG",
            marker=dict(
            size=2,
            opacity=0.8
        ),
            line=dict(width=10)
        )
    )

    # Ideal Point
    fig.add_trace(
        go.Scatter3d(
            x=[wholefront.population.objectives.max(axis=0)[0]],
            y=[wholefront.population.objectives.max(axis=0)[1]],
            z=[wholefront.population.objectives.max(axis=0)[2]],
            name="Ideal Point",
            marker=dict(
            size=4,
            opacity=0.8
        )
        )
    )

    fig.update_layout(
        scene = dict(
            xaxis = dict(range=[0,0.6],),
            yaxis = dict(range=[0,0.6],),
            zaxis = dict(range=[0,0.6],),
            aspectratio=dict(x=1,y=1,z=1))
                     )
    fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
    return fig
    